In [3]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-630b7fbc-ed09-4b29-912d-7c1458aa6b13;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 118ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/30 01:43:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/30 01:43:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/30 01:43:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from graphframes import GraphFrame

In [ ]:
window = Window.partitionBy("relationship").orderBy("timestamp")
df = readFirstEvents().withColumn("relationship", sort_array(array("actorID", "objectID")) )  \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1).cache()# \
                #.drop('rank').cache()

In [5]:
df = df.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))

In [6]:
df.count()

3756864

In [ ]:
df.limit(10).toPandas()

In [4]:
window = Window.partitionBy("relationship").orderBy("timestamp")
df = readFirstEvents().sort("timestamp").withColumn("relationship", sort_array(array("actorID", "objectID")) )  \
                .withColumn("row_num", row_number().over(window)) \
                .filter(col("row_num") == 1).cache()

 1:44AM UTC on Mar 30, 2023 --- read time: 7.198557138442993 seconds ---


In [ ]:
df_2.count()

In [9]:
df_23 = df.where((col('event_day') == 23) & (col('event_hour') == 14)
                & (col('event_minute') > 30)).cache()

In [10]:
df_23.drop('row_num','relationship')
df_23.count()

108062

In [11]:
df_23.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- relationship: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- row_num: integer (nul

In [ ]:
df_2 = df_2.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0)) \
        .drop('row_num','relationship')

In [ ]:
print("image path: "+str(df.select("image_path").distinct().count()))
print("image path: "+str(df.select("parent_image_path").distinct().count()))

In [ ]:
df_100k = df.limit(100000).cache()
df_2_100k = df_2.limit(100000).cache()

In [ ]:
df.count()

In [12]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_23.selectExpr('actorID as id').distinct()
dst_vertices = df_23.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [ ]:
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_2_100k.sort('timestamp').selectExpr('actorID as id').distinct()
dst_vertices = df_2_100k.sort('timestamp').selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_2_100k.sort('timestamp').selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

edges.sort('timestamp')

# Create GraphFrame
g_2 = GraphFrame(vertices, edges)

In [ ]:
# display the vertices
print("Vertices:")
g.vertices.show()

In [ ]:
# display the edges
print("Edges:")
g.edges.show(5)

In [ ]:
g.inDegrees.show(5)

In [13]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2_1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")
#motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6_1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [14]:
motifs2_1.count()

515764

In [15]:
motifs6_1.count()

29051144

In [23]:
filter_21 = motifs2_1.filter("e1.timestamp <= e2.timestamp")# and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")

In [24]:
filter_21.count()

334886

In [25]:
filter_61 = motifs6_1.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp and 
                                e3.timestamp <= e4.timestamp and e4.timestamp <= e5.timestamp and 
                                e5.timestamp <= e6.timestamp''')

In [26]:
filter_61.count()

2686166

In [30]:
filter_61 = transp_expl(filter_61).cache()

In [31]:
filter_61.limit(12).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,1fbee5f8-32b4-46c6-84b3-e4da0e508a52,55d3db82-c7be-46cf-8cec-953977555241,2019-09-23 14:44:28.257,PROCESS,OPEN,SysClient0851.systemia.com,None,None,%SystemRoot%\system32\csrss.exe,%SystemRoot%\system32\csrss.exe,None,None,None,None,None,None,0
1,1,e2,55d3db82-c7be-46cf-8cec-953977555241,4d7129e7-36c9-4b0a-8a65-c636fb7eee17,2019-09-23 14:45:20.644,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,\Device\HarddiskVolume1\Program Files (x86)\Go...,None,None,None,None,None,None,0
2,1,e3,4d7129e7-36c9-4b0a-8a65-c636fb7eee17,a5fe2d21-3b96-4cc1-97d2-50cc6b306bdd,2019-09-23 14:45:20.864,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\lsass...,\Device\HarddiskVolume1\Windows\system32\lsass...,None,None,None,None,None,None,0
3,1,e4,a5fe2d21-3b96-4cc1-97d2-50cc6b306bdd,b31c0f9d-1b6a-457f-a505-50fbf1f9a4bb,2019-09-23 14:45:41.982,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,\Device\HarddiskVolume1\Program Files (x86)\Go...,None,None,None,None,None,None,0
4,1,e5,b31c0f9d-1b6a-457f-a505-50fbf1f9a4bb,02b83f6a-375c-4f00-bfe6-de500101f5ba,2019-09-23 14:56:59.989,PROCESS,CREATE,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Windows\SYSTEM32\cmd.exe,\Device\HarddiskVolume1\Windows\system32\svcho...,None,None,None,None,None,None,0
5,1,e6,02b83f6a-375c-4f00-bfe6-de500101f5ba,a4a15731-4f90-480a-be3f-788887f2bbc9,2019-09-23 14:56:59.996,PROCESS,CREATE,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\SYSTEM32\cmd.exe,None,None,None,None,None,None,0
6,2,e1,137016c1-ce0c-4483-85f2-6b7d2c24d7cd,55d3db82-c7be-46cf-8cec-953977555241,2019-09-23 14:44:28.256,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,\Device\HarddiskVolume1\Program Files (x86)\Go...,None,None,None,None,None,None,0
7,2,e2,55d3db82-c7be-46cf-8cec-953977555241,4d7129e7-36c9-4b0a-8a65-c636fb7eee17,2019-09-23 14:45:20.644,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,\Device\HarddiskVolume1\Program Files (x86)\Go...,None,None,None,None,None,None,0
8,2,e3,4d7129e7-36c9-4b0a-8a65-c636fb7eee17,a5fe2d21-3b96-4cc1-97d2-50cc6b306bdd,2019-09-23 14:45:20.864,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\lsass...,\Device\HarddiskVolume1\Windows\system32\lsass...,None,None,None,None,None,None,0
9,2,e4,a5fe2d21-3b96-4cc1-97d2-50cc6b306bdd,b31c0f9d-1b6a-457f-a505-50fbf1f9a4bb,2019-09-23 14:45:41.982,PROCESS,OPEN,SysClient0851.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,\Device\HarddiskVolume1\Program Files (x86)\Go...,None,None,None,None,None,None,0


In [ ]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2_2 = g_2.find("(a)-[e1]->(b); (b)-[e2]->(c)")
#motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6_2 = g_2.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

In [ ]:
print(motifs2_2.count())
print(motifs6_2.count())

In [ ]:
#filter the motif2 to check the theory of timestamp increasing
cp3 = motifs2_2.filter("e1.timestamp < e2.timestamp")# and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")

In [ ]:
cp3.limit(2).toPandas()

In [ ]:
#explode it and see what you got
df_transp = cp3.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    ELSE 'e2' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col')

In [ ]:
df_onehot.limit(6).toPandas()

In [ ]:
# filter paths to only those where all edges are connected
cp_1 = motifs6_1.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src")
cp_2 = motifs6_2.filter('''e1.timestamp <= e2.timestamp and 
                            e2.timestamp <= e3.timestamp and 
                            e3.timestamp <= e4.timestamp and 
                            e4.timestamp <= e5.timestamp and 
                            e5.timestamp <= e6.timestamp''')

In [ ]:
print(cp_1.count())
print(cp_2.count())

In [ ]:
motifs6_2.limit(1).toPandas()

In [ ]:
cp_2.limit(12).toPandas()

Vertices:
+--------------------+
|                  id|
+--------------------+
|74fa9c45-fb97-447...|
|76269410-b376-416...|
|0844e602-c28c-4ff...|
|47be1d43-98e3-4d1...|
|08ca327d-85f6-4fe...|
|0a1c3d68-f678-46e...|
|0b840b3b-336e-460...|
|0bd4f4f1-7c85-4d5...|
|f7daa5be-d3e3-4c1...|
|b3c4e673-7662-456...|
|1134edf6-e2a4-41a...|
|12c5e76d-e26e-4eb...|
|8bd099cf-5bcc-437...|
|1604c531-2dfa-43f...|
|4ef22254-0006-437...|
|2baf6d0b-6e83-486...|
|25b541ee-4ae7-446...|
|99b2b14f-e990-4ee...|
|28731de7-1db6-492...|
|29289b94-46d8-49f...|
+--------------------+
only showing top 20 rows



In [29]:
#run transp explode on both and see what we have. first run it on both motif and then filter. this is on fully de-duped
#data.
%run trace_encode.ipynb

In [ ]:
te_2a = transp_expl(motifs6_2)
te_2b = transp_expl(cp_2)
#the results of the above are the same, no need to filter

Edges:
+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+--------------------+--------------------+--------+--------------------+---------+--------+-----------------+---------------+---------+
|                 src|                 dst|           timestamp| object|action|            hostname|user_name|privileges|          image_path|   parent_image_path|new_path|           file_path|direction|logon_id|requesting_domain|requesting_user|malicious|
+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+--------------------+--------------------+--------+--------------------+---------+--------+-----------------+---------------+---------+
|73d89b62-3424-41e...|0000e8e2-c112-461...|2019-09-24 14:24:...|   FILE|MODIFY|SysClient0419.sys...|     null|      null|              System|                null|    null|\Device\HarddiskV...|     null|    null|          

In [ ]:
te_2b.limit(12).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|0000e8e2-c112-461...|       1|
|00010fc7-3f0a-4d7...|       3|
|0004bb55-8f31-4c0...|       1|
|1ab5790a-9ca1-4e4...|       1|
|c5e98af6-4bbe-4b3...|       1|
+--------------------+--------+
only showing top 5 rows



In [9]:
# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

In [17]:
motifs2.count()

63245

In [18]:
#now run everything on all the data from 23Sep to see how long it takes
start_time = time.time()
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_23.selectExpr('actorID as id').distinct()
dst_vertices = df_23.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')
# Create GraphFrame
g = GraphFrame(vertices, edges)

#motifs
motifs6_23 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")
#end time
print("found connections: "+ str(time.time() - start_time))
#filter on time to generate final connected paths
cp = motifs6_23.filter('''e1.timestamp <= e2.timestamp and 
                            e2.timestamp <= e3.timestamp and 
                            e3.timestamp <= e4.timestamp and 
                            e4.timestamp <= e5.timestamp and 
                            e5.timestamp <= e6.timestamp''').cache()

#transpose and explode
te_23 = transp_expl(cp).cache()

#end time
print("elapsed time: "+ str(time.time() - start_time))

found connections: 1.3243827819824219


elapsed time: 2.5701472759246826


1322

In [10]:
te_23.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)



In [ ]:
# get the 6 edges in each path and convert to a pandas dataframe
connected_paths.select("e1").limit(1).collect()

In [ ]:
type(connected_paths)

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")

In [ ]:
connected_paths = spark.read.parquet(f"{s3_url_trusted}/prod/graph/test").cache()

In [ ]:
connected_paths.limit(5).toPandas()

In [ ]:
connected_paths.count()

In [ ]:
# using split and applying in ml with pypark
train, test, val = connected_paths.randomSplit([0.80, 0.15, 0.05], seed=12345)

In [ ]:
val.count()

In [ ]:
# create new dataframe to union and create duplicate rows
df_dup = df.withColumn('id', lit(None).cast(StringType()))

In [ ]:
df_dup.limit(5).toPandas()

In [ ]:
df_new = df.union(df_dup).sort("timestamp").cache()
df.unpersist()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
# len(df_new.select('relationship').distinct().collect())

In [ ]:
tot = df_new.count()

In [ ]:
df

In [ ]:
df_ratios = df.groupBy("object", "event_day", "malicious") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (col('cnt_per_group') / tot) * 100 ) \
              .sort("event_day", "malicious",  "perc_of_count_total", "object") \
              .cache()

In [ ]:
df_ratios.limit(20).toPandas()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
connected_paths.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test")